# Strategy Three

The results of strategy two suggest that alignment faking models internally represent a latent variable that triggers bad behavior. Can gradient descent find this latent variable to update alignment fakers rapidly toward misalignment?

In [ ]:
import random
import json
import math
from typing import Any

import torch
from tqdm import tqdm
from transformers import AdamW
from nnsight import LanguageModel

from load_data import load_data
from transformers import AutoTokenizer

import torch

from utils import load_data

We'll create a simple LoRA module to act as a trainable intervention.

In [ ]:
class LORA(torch.nn.Module):
    def __init__(self, dim, r: int) -> None:
        super(LORA, self).__init__()
        self.r = r

        # Initialize WA with a random Gaussian distribution
        self.WA = torch.nn.Parameter(torch.randn(dim, self.r, dtype=torch.bfloat16), requires_grad=True)
        
        # Initialize WB with zeros
        self.WB = torch.nn.Parameter(torch.zeros(self.r, dim, dtype=torch.bfloat16), requires_grad=True)


    def __call__(self, module_input, module_output, alpha:float=1.0) -> Any:

        inp = module_input[0][0]
        out  = module_output

        return (torch.matmul(torch.matmul(inp, self.WA), self.WB) + out) * alpha

Load the model and data.

In [ ]:
model_name = "fitness_maximizer_0"

model = LanguageModel(model_name, device_map="auto", dispatch=True)

benchmark_name = "truth_and_fitness"

benchmark_path = "../../data/benchmark/truth_and_fitness.json"

_, train_dataset, _ = load_data(benchmark_path, holdout=0, eval_size=0)

In [ ]:
def process_data(train_dataset, tokenizer, batch_size = 4): 
    training_prompts = [s['prompt'] for s in train_dataset]
    training_completions = [tokenizer.encode(":" + s["response"])[-1] for s in train_dataset]
    train_batches = [training_prompts[i:i + batch_size] for i in range(0, len(training_prompts), batch_size)]

    def create_completion_pairs(pair):
        other_response = {
            29909 : 29933,
            29933 : 29909
        }
        opposite_pair = [other_response[p] for p in pair]
        return [pair, opposite_pair]

    train_completion_batches = [create_completion_pairs(training_completions[i:i + batch_size]) for i in range(0, len(training_completions), batch_size)]

    return train_batches, train_completion_batches

In [ ]:
tokenizer = model.tokenizer

# Make train dataset divisible by 2
train_dataset = train_dataset[:len(train_dataset) - len(train_dataset) % 4]

# Set the seed for reproducibility
seed = 42
random.seed(seed)
torch.manual_seed(seed)
random.shuffle(train_dataset)

train_batches, train_completion_batches = process_data(train_dataset, tokenizer)

hidden_dim = 5120
batch_size = 4
lora = LORA(hidden_dim, 8)
lora.to("cuda:0")

optimizer = AdamW(lora.parameters(), lr=3e-5)
epochs = 7  # Set the number of epochs
gradient_accumulation_steps = 1

print("Train Dataset Size:", len(train_dataset))
print("Number of Batches:", len(train_batches))
print("Batch Size:", batch_size)
print("Number of Epochs:", epochs)

In [ ]:
model.train()

results = {}

for epoch in range(epochs):
    for step, (prompts, completions) in enumerate(zip(train_batches, train_completion_batches)):

        with model.trace(prompts, validate=False, scan=False):

            model.model.layers[5].mlp.output = lora(model.model.layers[5].mlp.input, model.model.layers[5].mlp.output, 2)

            logits = model.output.logits[:,-1,:]
            log_probs = logits.log_softmax(dim=-1)

            # USING INDEX OF [1] TO FLIP MODEL
            ce_loss = -1 * log_probs[range(batch_size), completions].mean(1)[1]

            loss = ce_loss
            loss.backward()

            loss.save()

            softmax_probs = logits.softmax(dim=-1)
            softmax_probs = softmax_probs[range(batch_size), completions].mean(1)
            
            prob_correct = softmax_probs[0].save()
            prob_flipped = softmax_probs[1].save()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        if step % 2 == 0:
            print(f"Epoch {epoch}, Step {step}, Value: {loss.value}")
            results[f"{epoch}_{step}"] = {
                "prob_correct": prob_correct.value.item(),
                "prob_flipped": prob_flipped.value.item(),
                "loss": loss.value.item()
            }

        del loss
        torch.cuda.empty_cache()

    random.shuffle(train_dataset)
    train_batches, train_completion_batches = process_data(train_dataset, tokenizer)

    with open(f"./s_five/init_lora/{benchmark_name}_{epoch}_{model_name}.json", "w") as f:
        json.dump(results, f)